In [1]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Jacob
[nltk_data]     Beel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
import pickle
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
  
lemmatizer = WordNetLemmatizer() 

def lemmatize(x):
    x = word_tokenize(x)
    x = [lemmatizer.lemmatize(i) for i in x]
    return ' '.join(x)

with open('data/dim_reduced_count_vector.pickle', 'rb') as f:
    reduced_data = pickle.load(f)
          
display(reduced_data.shape)

data = pd.read_csv('data/train.csv')
# data.comment_text = data.comment_text.apply(lemmatize)
display(data)

stops = stopwords.words('english')

vectorizer = TfidfVectorizer(max_df=0.95, min_df=5, stop_words=stops, ngram_range=(1, 3), max_features=10000)

(159571, 400)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [3]:
import numpy as np

def accuracy(y_pred, y):
    tp = np.sum(np.logical_and(y_pred == 1, y == 1))
    tn = np.sum(np.logical_and(y_pred == 0, y == 0))
    fp = np.sum(np.logical_and(y_pred == 1, y == 0))
    fn = np.sum(np.logical_and(y_pred == 0, y == 1))
    
    return tp, tn, fp, fn

y_pred = np.asarray([1, 1, 0])
y = np.asarray([1, 0, 1])

accuracy(y_pred, y)

(1, 0, 1, 1)

In [4]:
import pickle

with open('data/kmeans_assignments_tfidf.pickle', 'rb') as f:
    clusters = pickle.load(f)
    
print(clusters)
print(clusters.shape)

b = np.zeros((clusters.size, clusters.max() + 1))
b[np.arange(clusters.size), clusters] = 1
clusters = b
print(clusters.shape)

[10  0 10 ...  2  6  6]
(159571,)
(159571, 11)


In [5]:
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack, vstack

X = vectorizer.fit_transform(data.comment_text)
# X = hstack((X, clusters))
print(X.shape)
ids = data.id.values

y_layer_1 = data.toxic.values
y_layer_2 = data[['severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

test_split = 0.2
dev_split = 0.1
random_state = 42

X_train, X_test, y_train_1, y_test_1, y_train_2, y_test_2, id_train, id_test = train_test_split(
    X,
    y_layer_1, 
    y_layer_2,
    ids,
    test_size = 1 - (test_split + dev_split),
    random_state = random_state
)

X_dev, X_test, y_dev_1, y_test_1, y_dev_2, y_test_2, id_dev, id_test = train_test_split(
    X_test,
    y_test_1,
    y_test_2,
    id_test,
    test_size = test_split / (test_split + dev_split),
    random_state = random_state
)

(159571, 10000)


In [6]:
from classification_utils import *
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

layer_1 = LogisticRegression(C=5, solver='liblinear', max_iter=10000, class_weight='balanced')
layer_2_estimator = LogisticRegression(C=10, solver='liblinear', max_iter=10000, class_weight='balanced')
layer_2 = MultiOutputClassifier(layer_2_estimator)
layer_1.fit(X_train, y_train_1)
y_pred_layer_1 = layer_1.predict(X_test)
print("Layer 1 train score: ", layer_1.score(X_train, y_train_1))
print("Layer 1 test score: ", layer_1.score(X_test, y_test_1))

Layer 1 train score:  0.9657621524513798
Layer 1 test score:  0.9342527562544483


In [7]:
print(y_train_2[y_train_1 == 1].shape)
print(X_train[y_train_1 == 1].shape)

layer_2.fit(X_train[y_train_1 == 1], y_train_2[y_train_1 == 1])
print("Layer 2 train score: ", layer_2.score(X_train[y_train_1 == 1], y_train_2[y_train_1 == 1]))
y_pred_layer_2 = layer_2.predict(X_test[y_pred_layer_1 == 1])
print("Layer 2 test score: ", layer_2.score(X_test[y_pred_layer_1 == 1], y_test_2[y_pred_layer_1 == 1]))

(4636, 5)
(4636, 10000)
Layer 2 train score:  0.8237704918032787
Layer 2 test score:  0.5278826448063855


In [8]:
y_pred = np.zeros((y_pred_layer_1.shape[0], 6))
y_pred[:, 0] = y_pred_layer_1
y_pred[y_pred_layer_1 == 1, 1:] = y_pred_layer_2

y = np.hstack((y_test_1.reshape(-1, 1), y_test_2))

from sklearn.metrics import classification_report, accuracy_score

print('Accuracy: {:.2f}%'.format(accuracy_score(y, y_pred) * 100))
print(classification_report(y, y_pred, target_names= ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']))

Accuracy: 88.52%
               precision    recall  f1-score   support

        toxic       0.62      0.81      0.70      7083
 severe_toxic       0.35      0.55      0.43       761
      obscene       0.81      0.71      0.76      3935
       threat       0.36      0.49      0.41       222
       insult       0.65      0.63      0.64      3718
identity_hate       0.37      0.47      0.42       678

    micro avg       0.63      0.71      0.67     16397
    macro avg       0.53      0.61      0.56     16397
 weighted avg       0.65      0.71      0.67     16397
  samples avg       0.07      0.07      0.06     16397



C:\Users\Jacob Beel\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jacob Beel\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
y_train = np.hstack((y_train_1.reshape(-1, 1), y_train_2))
y_test = np.hstack((y_test_1.reshape(-1, 1), y_test_2))

layer_2.fit(X_train, y_train)
print(layer_2.score(X_train, y_train))
print(layer_2.score(X_test, y_test))
y_pred = layer_2.predict(X_test)

from sklearn.metrics import classification_report, accuracy_score

print('Accuracy: {:.2f}%'.format(accuracy_score(y_test, y_pred) * 100))
print(classification_report(y_test, y_pred, target_names= ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']))

0.933884815441499
0.8713658399022386
Accuracy: 87.14%
               precision    recall  f1-score   support

        toxic       0.62      0.80      0.70      7083
 severe_toxic       0.29      0.71      0.42       761
      obscene       0.68      0.81      0.74      3935
       threat       0.27      0.56      0.37       222
       insult       0.55      0.76      0.64      3718
identity_hate       0.28      0.57      0.37       678

    micro avg       0.56      0.78      0.65     16397
    macro avg       0.45      0.70      0.54     16397
 weighted avg       0.58      0.78      0.66     16397
  samples avg       0.06      0.07      0.06     16397



C:\Users\Jacob Beel\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jacob Beel\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
